<a href="https://colab.research.google.com/github/jobellet/detect_CS/blob/master/train_cs_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TRAIN A NETWORK

## First of all, drag all the files you want to use for training in the folder : uneye/work/TrainYourNetwork/

Ones all the files are uploaded, you can run the next cell

## The cell bellow authorizes google to modify files on your drive
- run the cell
- clic on the link
- select your account
- copy the code
- paste it in the empty field
- press enter

In [1]:
# Make your google drive accessible to colab
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


## The next cell installs all necessary dependencies



In [2]:
Path_to_UnEye_folder = '/gdrive/My Drive/uneye/' # You might need to change the path yourself

# to do math operations
import numpy as np
import scipy.io as io

# go to the uneye folder
import os
os.chdir(Path_to_UnEye_folder)

# to read .mat files
!pip install mat4py
import mat4py
# to do read .pkl files
import pandas as pd

# to do clustering
!pip install hdbscan

import uneye
from detect_CS import *

     |████████████████████████████████| 4.7MB 17.9MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for hdbscan: filename=hdbscan-0.8.26-cp36-cp36m-linux_x86_64.whl size=2361299 sha256=243480babe35196359b44c806c57dc0256cbfcd6630955b4335deafa66d05307
  Stored in directory: /root/.cache/pip/wheels/82/38/41/372f034d8abd271ef7787a681e0a47fc05d472683a7eb088ed
Successfully built hdbscan


# The next cell trains the network
## Change the variables of the next cell as described in the comments
## Then run the cell

In [0]:
# Set all the following parameters before running the cell
field_LFP = ['RAW']         # in .mat files where the LFP data is in the variable lfp then set: field_LFP = ['lfp']
                            # in .mat files where the LFP data is in the structure Data.lfp then set: field_LFP = ['Data','lfp']
                            # in .pkl file where the LFP data is in the key ['lfp'] then set: field_LFP = ['lfp']
                            # you can use either the LFP signal or the wide-band signal
field_high_pass = ['HIGH'] # same comment as above
field_label = ['Labels'] # same comment as above

# set the name of the weights of the network
weights_name = 'my_weight_name' # the weights is going to be saved in the folder /training

# set sample frequency of your recording
sampfreq = 25000 # in Hz

# size of the convolutional and maxpooling operations
ks = 9 # convolution kernel size 
mp = 7 # maxpooling size
# the number of bins (nb) taken into account by the network is given by
# nb = mp**2+mp**2*ks+(mp*ks)-mp+2*ks-2 


########################################################################################################################################################################################
training_folder = '/gdrive/My Drive/uneye/work/TrainYourNetwork/'
training_files = os.listdir(training_folder)

bigLFP = []
bigHIGH = []
bigLabels = []
for tf in training_files:
    LFP,High,Labels,_ = load_data(filename = training_folder+tf,field_LFP = field_LFP,field_high_pass = field_high_pass, field_label = field_label)
    interval_inspected = create_random_intervals(sampfreq,LFP,Labels)
    compLFP,compHIGH,compLabels = concatenate_segments(LFP,High,interval_inspected,Labels)
    bigLFP = np.concatenate((bigLFP,compLFP))
    bigHIGH = np.concatenate((bigHIGH,compHIGH))
    bigLabels = np.concatenate((bigLabels,compLabels))

model = uneye.DNN(max_iter = 3000, ks=ks,mp=mp, weights_name=weights_name,
                  sampfreq = sampfreq,augmentation = False,val_samples = 30, doDiff = False)
model.train(bigLFP,bigHIGH,bigLabels)